## VN2 Inventory simulation code
Réplica - entendimiento del código original

Código "oficial": https://www.datasource.ai/en/users/jose-ignacio-ortega-labra/competitions/vn2-inventory-planning-challenge/datathon_detail/notebook/27/show

### 1. Código original

In [1]:
def update_inventory_state(player="Nicolas", order=1): 

    sales = pd.read_csv("Data/Not For Share - Sales - All.csv").set_index(INDEX) 
#Contains all the sales data.
    date, previous_date  = compute_dates(order)
    demand = sales[date]
    
    if order < 7:
        
# Participants need to generate a order
        orders = extract_order(sales, player=player, order=order)
    else:
        
# We keep on running the simulation for two extra rounds without further inputs from participants.
        
# We can do this because of the lead time
        orders = pd.DataFrame(index=sales.index, columns=['Orders'], data=0)

    previous_state = pd.read_csv(f"Data/{player}/Week {order-1} - {previous_date} - State.csv").set_index(INDEX)   

    state = pd.DataFrame(index=sales.index)
    state["Start Inventory"] = previous_state["End Inventory"] + previous_state["In Transit W+1"]
    state["Sales"] = state["Start Inventory"].clip(upper=demand)
    state["Missed Sales"] = demand - state["Sales"]
    state["End Inventory"] = state["Start Inventory"] - state["Sales"]
    state["In Transit W+1"] = previous_state["In Transit W+2"]
    state["In Transit W+2"] = orders
    state["Holding Cost"] = state["End Inventory"]*HOLDING_COST 
    state["Shortage Cost"] = state["Missed Sales"]*SHORTAGE_COST 
    state["Cumulative Holding Cost"] = previous_state["Cumulative Holding Cost"] + state["Holding Cost"]
    state["Cumulative Shortage Cost"] = previous_state["Cumulative Shortage Cost"] + state["Shortage Cost"]

    state.to_csv(f"Data/{player}/Week {order} - {date} - State.csv") 
#State at the end of the week.

    round_cost = state[["Holding Cost","Shortage Cost"]].sum().sum()    
    cumulative_cost = state[["Cumulative Holding Cost","Cumulative Shortage Cost"]].sum().sum()

    print(f"Week {order} End\t\tState:\t{round_cost.round(1)}\tCumulative: {cumulative_cost.round(1)}")

    return round_cost, cumulative_cost

### 2. Entendimiento código

In [3]:
import pandas as pd
import numpy as np

In [8]:
# set root repo

import subprocess
from pathlib import Path
import os

repo_root = subprocess.run(
    ["git", "rev-parse", "--show-toplevel"], capture_output=True, text=True
).stdout.strip()

ROOT = Path(repo_root)
os.chdir(ROOT)
print(ROOT)

/Users/joseortega/Documents/GitHub/vn2_challenge


In [9]:
# params GLOBAL
HOLDING_COST = 0.2
SHORTAGE_COST = 1

In [22]:
# params - input de la función original
player="benchmark"
order=1 # quizás para la semana 0, el order es cero order=0

##### replicar código original para entender qué hace

In [13]:
# código original - leer sales semana 0
folder_data = "data/input"
INDEX = ["Store", "Product"]

path_sales = f"{folder_data}/Week 0 - 2024-04-08 - Sales.csv"
sales = pd.read_csv(path_sales).set_index(INDEX)
sales.columns = pd.to_datetime(sales.columns)

sales.head()

2021-04-12  2021-04-19  2021-04-26  2021-05-03  2021-05-10  \
Store Product                                                               
0     126             0.0         0.0         3.0         3.0         0.0   
      182             0.0         0.0         0.0         0.0         0.0   
1     124            13.0         4.0        10.0         5.0         1.0   
2     124             5.0         5.0        12.0        16.0        10.0   
      126             0.0         0.0         6.0         5.0         7.0   

               2021-05-17  2021-05-24  2021-05-31  2021-06-07  2021-06-14  \
Store Product                                                               
0     126             1.0         1.0         0.0         1.0         0.0   
      182             0.0         0.0         0.0         0.0         0.0   
1     124             2.0         3.0         4.0         2.0         1.0   
2     124             8.0        10.0         9.0         7.0        10.0   
      126             4.0         1.0         2.0         6.0         5.0   

               ...  2024-02-05  2024-02-12  2024-02-19  2024-02-26  \
Store Product  ...                                                   
0     126      ...         0.0         2.0         2.0         0.0   
      182      ...         1.0         1.0         0.0         0.0   
1     124      ...         8.0        17.0         6.0        11.0   
2     124      ...         6.0         8.0         8.0        18.0   
      126      ...         2.0         0.0         2.0         5.0   

               2024-03-04  2024-03-11  2024-03-18  2024-03-25  2024-04-01  \
Store Product                                                               
0     126             0.0         0.0         0.0         0.0         2.0   
      182             1.0         1.0         0.0         3.0         1.0   
1     124             8.0        12.0         6.0         7.0         9.0   
2     124            11.0        14.0         3.0        12.0        13.0   
      126             2.0         2.0         4.0         2.0         3.0   

               2024-04-08  
Store Product              
0     126             2.0  
      182             1.0  
1     124             7.0  
2     124             2.0  
      126             2.0  

[5 rows x 157 columns]

In [20]:
# código original - se obtiene la fecha actual y la fecha previa - se filtra la demanda para esa fecha
#date, previous_date  = compute_dates(order)
#demand = sales[date]

date = sales.columns[-1]
previous_date = sales.columns[-2]

demand = sales[date]

print("date: ", date)
print("previous_date: ", previous_date)

date:  2024-04-08 00:00:00
previous_date:  2024-04-01 00:00:00


In [ ]:
# código original - generar la orden
# OJO IMPORTANTE: se corre la simulación por 2 semanas sin cambios en el input, es decir, si la demanda varía mucho en 
# esas 2 próximas semanas, va a haber un incremento en el costo.
# TODO: revisar si se está entendiendo bien

# if order < 7:
# # Participants need to generate a order
#     orders = extract_order(sales, player=player, order=order)
# else:
# # We keep on running the simulation for two extra rounds without further inputs from participants.
# # We can do this because of the lead time
#     orders = pd.DataFrame(index=sales.index, columns=['Orders'], data=0)


# en mi caso, voy a leer el archivo de ordenes template
path_submission = f"{folder_data}/Week 0 - Submission Template.csv"
orders = pd.read_csv(path_submission).set_index(INDEX)
orders.head()

0
Store Product   
0     126      0
      182      0
1     124      0
2     124      0
      126      0

In [26]:
# código original - leer el estado de la fecha anterior. En este caso sería la fecha 0 (para la entrega de la semana 1)
# previous_state = pd.read_csv(f"Data/{player}/Week {order-1} - {previous_date} - State.csv").set_index(INDEX)

# mi caso - voy a leer state de la semana 0
# VOY A SIMULAR QUE LA ORDER ENTREGADO Y LA FECHA ENTREGADA CORRESPONDE A LA INFO DE LA SEMANA 1
path_state = f"{folder_data}/Week 0 - 2024-04-08 - Initial State.csv"
previous_state = pd.read_csv(path_state).set_index(INDEX)
previous_state.head(3) # state de la semana 0

Start Inventory  Sales  Missed Sales  End Inventory  \
Store Product                                                        
0     126                    0      0             0              3   
      182                    0      0             0              1   
1     124                    0      0             0              6   

               In Transit W+1  In Transit W+2  Holding Cost  Shortage Cost  \
Store Product                                                                
0     126                   0               3             0              0   
      182                   0               1             0              0   
1     124                   0               6             0              0   

               Cumulative Holding Cost  Cumulative Shortage Cost  
Store Product                                                     
0     126                            0                         0  
      182                            0                         0  
1     124                            0                         0

In [29]:
# # código original - generar ARCHIVO STATE PARA LA SEMANA 1 (SEMANA DE LA ENTREGA)
# state = pd.DataFrame(index=sales.index)
# state["Start Inventory"] = previous_state["End Inventory"] + previous_state["In Transit W+1"]
# state["Sales"] = state["Start Inventory"].clip(upper=demand)
# state["Missed Sales"] = demand - state["Sales"]
# state["End Inventory"] = state["Start Inventory"] - state["Sales"]
# state["In Transit W+1"] = previous_state["In Transit W+2"]
# state["In Transit W+2"] = orders
# state["Holding Cost"] = state["End Inventory"]*HOLDING_COST 
# state["Shortage Cost"] = state["Missed Sales"]*SHORTAGE_COST 
# state["Cumulative Holding Cost"] = previous_state["Cumulative Holding Cost"] + state["Holding Cost"]
# state["Cumulative Shortage Cost"] = previous_state["Cumulative Shortage Cost"] + state["Shortage Cost"]

# state.to_csv(f"Data/{player}/Week {order} - {date} - State.csv") 
# #State at the end of the week.


# en mi caso - entender cómo se genera el STATE para poder ver cómo se calcula todo - SIGUIENTE LÍNEAS DE CÓDIGO - SE HACE 1 POR 1

In [59]:
# generar índices
state = pd.DataFrame(index=sales.index)
state.head(3)

Empty DataFrame
Columns: []
Index: [(0, 126), (0, 182), (1, 124)]

In [60]:
# calcular inventario inicial para la semana 1
# sumar el inventario con el que terminó la semana 0 y sumar el inventario que va a llegar en la semana +1

# OJO: POR ESO EN EL STATE DE LA SEMANA 0 NO HAY INFO DE START INVENTORY, SALES, ETC, PORQUE PARA GENERAR EL STATE DE LA SEMANA 1 ESO NO ES NECESARIO
# ASÍ QUE NO SE AGREGÓ LA INFO
state["Start Inventory"] = previous_state["End Inventory"] + previous_state["In Transit W+1"]
state.head(3)

Start Inventory
Store Product                 
0     126                    3
      182                    1
1     124                    6

In [61]:
# ventas: es el invetario clipeado por arriba por la demanda. 
# osea si el inventario es mayor a la demanda, se vende toda la demanda
# por otro lado si el inventario es menor a la demanda, se vende hasta lo que alacanzó de inventario
state["Sales"] = state["Start Inventory"].clip(upper=demand)
state.head(3)

Start Inventory  Sales
Store Product                        
0     126                    3      2
      182                    1      1
1     124                    6      6

In [62]:
demand.head(3)

Store  Product
0      126        2.0
       182        1.0
1      124        7.0
Name: 2024-04-08 00:00:00, dtype: float64

In [63]:
# DUDA: EN LA DATA SALES, CUANDO NO HAY STOCK LA VENTA ES CERO, PERO ESO NO ESTÁ ESTIMANDO DE FORMA CORRECTA LA DEMANDA POTENCIAL QUE PUDO HABER HABIDO

In [64]:
# calcular ventas perdidas - 
# # pero esto sigue siendo el problema, que INCLUSO EN LOS DATOS REALES, CUANDO SE QUEDAN SIN STOCK SE NECESITA ESTIMAR ESA VENTA POTENCIAL
# PERDIDA

# ej en los casos que se quedó sin stock, el inventario inicial era cero, por lo que las ventas eran cero (estimación de demanda errónea)
# entonces la demanda ya es cero y las ventas son cero, así nunca hay costo de missed sales
state["Missed Sales"] = demand - state["Sales"]

In [65]:
# inventario final = inventario inicial - ventas
state["End Inventory"] = state["Start Inventory"] - state["Sales"]

In [66]:
# en transito para semana 1, es lo que estaba en tránsito en semana 2 en el state de la semana pasada
state["In Transit W+1"] = previous_state["In Transit W+2"]

In [67]:
# EN TRÁNSITO QUE VA A LLEGAR EN LA SEMANA 2, ES LO QUE SE DECIDIÓ PEDIR ESTA SEMANA
state["In Transit W+2"] = orders

state.head(3)

Start Inventory  Sales  Missed Sales  End Inventory  \
Store Product                                                        
0     126                    3      2           0.0              1   
      182                    1      1           0.0              0   
1     124                    6      6           1.0              0   

               In Transit W+1  In Transit W+2  
Store Product                                  
0     126                   3               0  
      182                   1               0  
1     124                   6               0

In [68]:
# HOLDING COST
# el inventario que quedó (que se sobre estimó la demanda)
# YO: PERO TODO ESTO DEPENDE DE QUÉ TAN PRECISO ES MI FORECAST
state["Holding Cost"] = state["End Inventory"]*HOLDING_COST

In [69]:
# SHORTAGE COST
# las ventas perdidas por subestimar
state["Shortage Cost"] = state["Missed Sales"]*SHORTAGE_COST 

In [70]:
# costos acumulados de las decisiones anteriores
state["Cumulative Holding Cost"] = previous_state["Cumulative Holding Cost"] + state["Holding Cost"]
state["Cumulative Shortage Cost"] = previous_state["Cumulative Shortage Cost"] + state["Shortage Cost"]

state.head(3)

Start Inventory  Sales  Missed Sales  End Inventory  \
Store Product                                                        
0     126                    3      2           0.0              1   
      182                    1      1           0.0              0   
1     124                    6      6           1.0              0   

               In Transit W+1  In Transit W+2  Holding Cost  Shortage Cost  \
Store Product                                                                
0     126                   3               0           0.2            0.0   
      182                   1               0           0.0            0.0   
1     124                   6               0           0.0            1.0   

               Cumulative Holding Cost  Cumulative Shortage Cost  
Store Product                                                     
0     126                          0.2                       0.0  
      182                          0.0                       0.0  
1     124                          0.0                       1.0

In [72]:
# state.to_csv(f"Data/{player}/Week {order} - {date} - State.csv") 
#State at the end of the week.

In [74]:
# costo total de la semana 1
round_cost = state[["Holding Cost","Shortage Cost"]].sum().sum()
round_cost

np.float64(381.8)

In [75]:
# costo acumulado hasta ahora
cumulative_cost = state[["Cumulative Holding Cost","Cumulative Shortage Cost"]].sum().sum()
cumulative_cost

np.float64(381.8)

In [76]:
print(f"Week {order} End\t\tState:\t{round_cost.round(1)}\tCumulative: {cumulative_cost.round(1)}")

Week 1 End		State:	381.8	Cumulative: 381.8
